In [13]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path.append('../Viterbi')
from CurvatureTools import *
from SyntheticCurves import *
from Viterbi import getCSM

In [25]:
t = np.linspace(0, 1, 1000)
X = get2DFigure8(t)
earcon = np.array([0.5, 0.5])
dists = getCSM(X, earcon[None, :]).flatten()

V = getCurvVectors(X, 1, 5, loop=True)[1]

VMag = np.sqrt(np.sum(V**2, axis=1))


plt.figure(figsize=(6, 6))
plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], c=t)
plt.scatter([earcon[0]], [earcon[1]], c='C2')
plt.title("Curve")

plt.subplot(222)
plt.scatter(t, dists, c=t)
plt.title("Distance to earcon")


plt.subplot(223)
V = V*(1+0.3*t[:, None])
plt.scatter(V[:, 0], V[:, 1], c=t)
plt.scatter([0], [0], c='C1')
plt.plot([-0.02, 0.02], [0, 0], c='C1')
plt.plot([0, 0], [-0.03, 0.03], c='C1')
plt.title("Velocity")

plt.subplot(224)
plt.scatter(t, VMag, c=t)
plt.title("Speed")

plt.show()
